In [1]:
# 'scwrl4 -i {pdb_filename} -s {new_fasta_seq_file} -o {remodeled_pdb_file} > {workSubfolder}/modeling.log' 

In [2]:
import os
import pandas as pd

In [3]:
# Load Data
df = ''

In [4]:
import os

# Assuming `df` is your DataFrame
def create_bash_script(df, script_path):
    with open(script_path, 'w') as modelScript:
        # Write the initial file check lines
        fileCheckString = ("#############################################################################################################################\n"
                           "############ Checking for scwrl4 in the workFolder ##########################################################################\n"
                           "#############################################################################################################################\n"
                           "echo \"Checking for scwrl4..\"; if [ ! -f \"../src/scwrl/scwrl4\" ]; then echo \"[ERROR]: scwrl4 not found. Exitting\!\"; exit 1; else echo \"scwrl4 found. Proceeding....\"; fi ; \n")
        modelScript.write(fileCheckString)

        # Loop through the DataFrame
        for index, row in df.iterrows():
            cath_id = row['cath_id']
            sequences = row['sequences']
            
            workSubfolder = f"../data/{cath_id}"
            
            pdb_dir = f"{workSubfolder}/pdb"
            pdb_filename = f"{pdb_dir}/{cath_id}"
            
            seqs_dir = f"{workSubfolder}/seqs"
            new_fasta_seq_file = f"{seqs_dir}/modified.fasta"

            # Ensure the directories exist
            os.makedirs(pdb_dir, exist_ok=True)
            os.makedirs(seqs_dir, exist_ok=True)

            # Write sequences to modified.fasta
            with open(new_fasta_seq_file, 'w') as fasta_file:
                fasta_file.write(sequences)

            # Prepare script lines
            commentString = ("#############################################################################################################################\n"
                             f"###################### BUILDING SCWRL MODEL FOR WORK INDEX {index} (pdb: {cath_id}) ########################################\n"
                             "#############################################################################################################################\n")
            modelScript.write(commentString)

            checkLogsForSkip = f"( if ! {{ [ -f {seqs_dir}/step1-model.log ] && ( grep -q Scwrl4_Execution {seqs_dir}/step1-model.log ) }} ; then   "
            runScwrl = f" ../src/scwrl/scwrl4  -i {pdb_filename} -s {new_fasta_seq_file} -o {pdb_dir}/{cath_id}_remodeled.pdb > {workSubfolder}/modeling.log; "
            addRunToLogs = f" echo \"scwrl4 model built for [{cath_id}] on $(date) \" > {workSubfolder}/modelStatus.log ;  fi; ) &"
            newLine = checkLogsForSkip + runScwrl + addRunToLogs + "\n"
            statusString = f"echo \"[FINISHED]: {index} Modeled PDB: {cath_id}\"\n"

            modelScript.write(newLine)
            modelScript.write(statusString)
            modelScript.write("wait;\n")

In [5]:
# Create the bash script
bash_script = 'scwrl4_remodeling.sh'
create_bash_script(df, bash_script)

AttributeError: 'str' object has no attribute 'iterrows'